In [1]:
import sys
sys.path.append('..')
from MPRA_predict.utils import *

In [2]:
# MPRA_df = pd.read_csv('../data/Gosai_MPRA/Gosai_MPRA_seq_feature_alphagenome_log1p.csv')
MPRA_df = pd.read_csv('../data/Gosai_MPRA/Gosai_MPRA_seq_feature_alphagenome_log1p_zscore.csv')

MPRA_df

,seq,ID,chr,pos,ref_allele,alt_allele,allele,OL,data_project,K562,HepG2,SK-N-SH,A549,HCT116,K562_DNase,K562_H3K4me3,K562_H3K27ac,K562_CTCF,HepG2_DNase,HepG2_H3K4me3,HepG2_H3K27ac,HepG2_CTCF,SK-N-SH_DNase,SK-N-SH_H3K4me3,SK-N-SH_H3K27ac,SK-N-SH_CTCF,HCT116_DNase,HCT116_H3K4me3,HCT116_H3K27ac,HCT116_CTCF,A549_DNase,A549_H3K4me3,A549_H3K27ac,A549_CTCF
0,AGTGCTGGTTCCGTCACCCCCTCCCAAGGAAGTAGGTCTGAGCAGC...,1:14677:G:A:A:wC,chr1,14677,G,A,alt,OL41_42,GTEx,0.014,-0.054,0.012,NaN,-0.709,0.027,2.360,1.869,NaN,0.024,1.872,1.688,NaN,0.025,1.597,0.904,NaN,0.027,2.079,2.661,NaN,0.016,2.169,1.794,NaN
1,AGTGCTGGTTCCGTCACCCCCTCCCAAGGAAGTAGGTCTGAGCAGC...,1:14677:G:A:R:wC,chr1,14677,G,A,ref,OL41_42,GTEx,-1.042,0.183,0.094,NaN,-0.266,0.029,2.415,1.935,NaN,0.026,1.923,1.755,NaN,0.029,1.667,0.962,NaN,0.029,2.151,2.759,NaN,0.018,2.231,1.860,NaN
2,CATACATTTTTATTCTGGTCACTGTCCAACGACATTCCTCAAATGA...,1:63697:T:C:A:wC,chr1,63697,T,C,alt,OL41_42,GTEx,-1.004,0.640,-0.188,NaN,0.619,0.018,1.765,1.896,NaN,0.023,2.019,1.688,NaN,0.004,1.937,1.555,NaN,0.024,2.056,2.004,NaN,0.010,2.142,1.825,NaN
3,CATACATTTTTATTCTGGTCACTGTCCAACGACATTCCTCAAATGA...,1:63697:T:C:R:wC,chr1,63697,T,C,ref,OL41_42,GTEx,-0.779,0.751,0.274,NaN,0.668,0.018,1.776,1.907,NaN,0.022,1.990,1.699,NaN,0.004,1.987,1.626,NaN,0.024,2.056,2.027,NaN,0.010,2.211,1.950,NaN
4,AATAGACTCTAGTACTCTGTATTATGCAAAATTTGTCTATGTTACA...,1:64764:C:T:R:wC,chr1,64764,C,T,ref,OL41_42,GTEx,-0.457,-0.645,0.061,NaN,-0.168,0.006,1.253,1.538,NaN,0.013,1.676,1.469,NaN,0.004,1.568,1.164,NaN,0.014,1.622,1.776,NaN,0.008,1.812,1.181,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
760674,CTGCATCTGTGCAGCCTGGCAGCGGCGGCGCTGCGCTGTGACATTT...,Y:15815629:NA:NA,chrY,15815629,NaN,NaN,NaN,OL15,OL15,0.708,1.294,1.077,NaN,NaN,0.152,3.474,2.860,NaN,0.078,2.908,2.596,NaN,0.057,2.786,2.158,NaN,0.110,3.256,3.747,NaN,0.041,3.162,2.626,NaN
760675,GCACTTCACATTTGTTTTTAGGGTTACATAGTCTACTCTGTATCCT...,Y:21740684:NA:NA,chrY,21740684,NaN,NaN,NaN,OL15,OL15,-0.708,-0.547,-0.429,NaN,NaN,0.014,1.409,1.669,NaN,0.012,1.472,1.346,NaN,0.005,1.200,0.783,NaN,0.015,1.467,1.794,NaN,0.007,1.637,1.174,NaN
760676,TGTTCAAGTGGCCACAGGGTTACTTGCTTTAGCATGGCTCCTTGGC...,Y:21740884:NA:NA,chrY,21740884,NaN,NaN,NaN,OL15,OL15,-0.146,-0.200,-0.310,NaN,NaN,0.021,1.622,1.896,NaN,0.026,1.802,1.676,NaN,0.006,1.556,1.363,NaN,0.031,1.919,2.676,NaN,0.014,2.395,2.970,NaN
760677,TAATTAGTTGGGAAGGTTCAGGTTCTGGGACATCCGTTGCTATTTC...,Y:21741084:NA:NA,chrY,21741084,NaN,NaN,NaN,OL15,OL15,-0.119,-0.090,-0.387,NaN,NaN,0.015,1.563,1.741,NaN,0.015,1.614,1.467,NaN,0.005,1.376,1.020,NaN,0.020,1.685,1.950,NaN,0.007,1.724,1.202,NaN


In [3]:
cell_types = ['K562', 'HepG2', 'SK-N-SH', 'HCT116', 'A549']
assays = ['DNase', 'H3K4me3', 'H3K27ac', 'CTCF']

In [4]:
data_split_dict = {}
data_split_dict['total'] = np.ones(len(MPRA_df), dtype=bool)

data_split_dict['chrom'] = MPRA_df['chr'].isin([f'chr{i}' for i in (list(range(1, 23)) + ['X', 'Y'])])
data_split_dict['designed'] = ~data_split_dict['chrom']

data_split_dict['train'] = MPRA_df['chr'].isin([f'chr{i}' for i in (list(range(3, 23)) + ['X', 'Y'])])
data_split_dict['valid'] = MPRA_df['chr'].isin(['chr1'])
data_split_dict['test'] = MPRA_df['chr'].isin(['chr2'])

data_split_dict['ref'] = (MPRA_df['allele'] == 'ref') | (MPRA_df['allele'].isna())
data_split_dict['alt'] = MPRA_df['allele'] == 'alt'



# 计算每条序列在不同细胞类型中的最大值和最小值之差
MPRA_df['range'] = MPRA_df[cell_types].max(axis=1, skipna=True) - MPRA_df[cell_types].min(axis=1, skipna=True)
# 计算 range 的第95百分位
threshold = np.percentile(MPRA_df['range'].dropna(), 95)
# 前5%的高差值序列为 diff，其余为 const
data_split_dict['diff'] = MPRA_df['range'] > threshold
data_split_dict['const'] = ~data_split_dict['diff']



for key in data_split_dict:
    print(key, data_split_dict[key].sum())

keys = list(data_split_dict.keys())
for k1 in keys:
    for k2 in keys:
        data_split_dict[f'{k1}+{k2}'] = data_split_dict[k1] & data_split_dict[k2]

total 760679
chrom 760679
designed 0
train 631073
valid 70952
test 58654
ref 389266
alt 371413
diff 38034
const 722645


In [ ]:
MPRA_df = pd.read_csv('../data/Gosai_MPRA/Gosai_MPRA_seq_feature_alphagenome_log1p.csv')

In [5]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, HistGradientBoostingRegressor

In [9]:
cell_types = ['K562', 'HepG2', 'SK-N-SH', 'HCT116', 'A549']
assays = ['DNase', 'H3K4me3', 'H3K27ac']
data_splits = ['train', 'test', 'test+diff']


# for train_cell_types in [['K562'], ['HepG2'], ['SK-N-SH'], ['K562', 'HepG2'], ['K562', 'HepG2', 'SK-N-SH']]:
for train_cell_types in [['K562'], ['K562', 'HepG2'], ['K562', 'HepG2', 'SK-N-SH']]:
    # train
    train_data_split = 'train'

    X_train = []
    y_train = []
    for cell_type in train_cell_types:
        condition = data_split_dict[train_data_split]
        X_columns = [f'{cell_type}_{assay}' for assay in assays]
        y_column = cell_type
        
        X_train.append(MPRA_df.loc[condition, X_columns])
        y_train.append(MPRA_df.loc[condition, y_column])

    X_train = np.concatenate(X_train, axis=0)
    y_train = np.concatenate(y_train, axis=0)
    print(X_train.shape, y_train.shape)


    model = LinearRegression()
    # model = MLPRegressor(hidden_layer_sizes=100, alpha=1e-3)
    model.fit(X_train, y_train)


    # valid
    pearson_df = pd.DataFrame(dtype=float)

    for cell_type in cell_types:
        for data_split in data_splits:
            condition = data_split_dict[data_split]
            X_columns = [f'{cell_type}_{assay}' for assay in assays]
            y_column = cell_type

            X_valid = np.array(MPRA_df.loc[condition, X_columns])
            y_valid = np.array(MPRA_df.loc[condition, y_column])

            y_pred = model.predict(X_valid)

            r = pearson(y_valid, y_pred)[0]
            pearson_df.loc[cell_type, data_split] = r

    print(f'trained on {train_cell_types}')
    print(f'model architecture: LinearRegression()')
    print(pearson_df)

(631073, 3) (631073,)
trained on ['K562']
model architecture: LinearRegression()
         train  test  test+diff
K562     0.616 0.589      0.601
HepG2    0.534 0.505      0.555
SK-N-SH  0.571 0.532      0.470
HCT116   0.577 0.560      0.567
A549     0.453 0.416      0.410
(1262146, 3) (1262146,)
trained on ['K562', 'HepG2']
model architecture: LinearRegression()
         train  test  test+diff
K562     0.614 0.587      0.601
HepG2    0.535 0.504      0.562
SK-N-SH  0.564 0.524      0.469
HCT116   0.581 0.561      0.570
A549     0.448 0.414      0.406
(1893219, 3) (1893219,)
trained on ['K562', 'HepG2', 'SK-N-SH']
model architecture: LinearRegression()
         train  test  test+diff
K562     0.612 0.584      0.602
HepG2    0.537 0.503      0.563
SK-N-SH  0.560 0.519      0.464
HCT116   0.581 0.560      0.571
A549     0.444 0.408      0.403


In [10]:
cell_types = ['K562', 'HepG2', 'SK-N-SH', 'HCT116', 'A549']
assays = ['DNase', 'H3K4me3', 'H3K27ac']
data_splits = ['train', 'test', 'test+diff']


# for train_cell_types in [['K562'], ['HepG2'], ['SK-N-SH'], ['K562', 'HepG2'], ['K562', 'HepG2', 'SK-N-SH']]:
for train_cell_types in [['K562'], ['K562', 'HepG2'], ['K562', 'HepG2', 'SK-N-SH']]:
    # train
    train_data_split = 'train'

    X_train = []
    y_train = []
    for cell_type in train_cell_types:
        condition = data_split_dict[train_data_split]
        X_columns = [f'{cell_type}_{assay}' for assay in assays]
        y_column = cell_type
        
        X_train.append(MPRA_df.loc[condition, X_columns])
        y_train.append(MPRA_df.loc[condition, y_column])

    X_train = np.concatenate(X_train, axis=0)
    y_train = np.concatenate(y_train, axis=0)
    print(X_train.shape, y_train.shape)

    model = MLPRegressor(hidden_layer_sizes=20, alpha=1e-3)
    model.fit(X_train, y_train)


    # valid
    pearson_df = pd.DataFrame(dtype=float)

    for cell_type in cell_types:
        for data_split in data_splits:
            condition = data_split_dict[data_split]
            X_columns = [f'{cell_type}_{assay}' for assay in assays]
            y_column = cell_type

            X_valid = np.array(MPRA_df.loc[condition, X_columns])
            y_valid = np.array(MPRA_df.loc[condition, y_column])

            y_pred = model.predict(X_valid)

            r = pearson(y_valid, y_pred)[0]
            pearson_df.loc[cell_type, data_split] = r

    print(f'trained on {train_cell_types}')
    print(f'model architecture: MLPRegressor()')
    print(pearson_df)

(631073, 3) (631073,)
trained on ['K562']
model architecture: MLPRegressor()
         train  test  test+diff
K562     0.660 0.628      0.645
HepG2    0.552 0.521      0.605
SK-N-SH  0.268 0.197      0.244
HCT116   0.553 0.528      0.586
A549     0.106 0.040      0.257
(1262146, 3) (1262146,)
trained on ['K562', 'HepG2']
model architecture: MLPRegressor()
         train  test  test+diff
K562     0.655 0.624      0.636
HepG2    0.586 0.556      0.626
SK-N-SH  0.362 0.306      0.377
HCT116   0.582 0.560      0.612
A549     0.246 0.184      0.367
(1893219, 3) (1893219,)
trained on ['K562', 'HepG2', 'SK-N-SH']
model architecture: MLPRegressor()
         train  test  test+diff
K562     0.638 0.607      0.610
HepG2    0.581 0.549      0.606
SK-N-SH  0.588 0.548      0.507
HCT116   0.604 0.582      0.611
A549     0.446 0.412      0.416


In [11]:
MPRA_df = pd.read_csv('../data/Gosai_MPRA/Gosai_MPRA_seq_feature_alphagenome_log1p_zscore.csv')

In [12]:
cell_types = ['K562', 'HepG2', 'SK-N-SH', 'HCT116', 'A549']
assays = ['DNase', 'H3K4me3', 'H3K27ac']
data_splits = ['train', 'test', 'test+diff']


# for train_cell_types in [['K562'], ['HepG2'], ['SK-N-SH'], ['K562', 'HepG2'], ['K562', 'HepG2', 'SK-N-SH']]:
for train_cell_types in [['K562'], ['K562', 'HepG2'], ['K562', 'HepG2', 'SK-N-SH']]:
    # train
    train_data_split = 'train'

    X_train = []
    y_train = []
    for cell_type in train_cell_types:
        condition = data_split_dict[train_data_split]
        X_columns = [f'{cell_type}_{assay}' for assay in assays]
        y_column = cell_type
        
        X_train.append(MPRA_df.loc[condition, X_columns])
        y_train.append(MPRA_df.loc[condition, y_column])

    X_train = np.concatenate(X_train, axis=0)
    y_train = np.concatenate(y_train, axis=0)
    print(X_train.shape, y_train.shape)


    model = LinearRegression()
    # model = MLPRegressor(hidden_layer_sizes=100, alpha=1e-3)
    model.fit(X_train, y_train)


    # valid
    pearson_df = pd.DataFrame(dtype=float)

    for cell_type in cell_types:
        for data_split in data_splits:
            condition = data_split_dict[data_split]
            X_columns = [f'{cell_type}_{assay}' for assay in assays]
            y_column = cell_type

            X_valid = np.array(MPRA_df.loc[condition, X_columns])
            y_valid = np.array(MPRA_df.loc[condition, y_column])

            y_pred = model.predict(X_valid)

            r = pearson(y_valid, y_pred)[0]
            pearson_df.loc[cell_type, data_split] = r

    print(f'trained on {train_cell_types}')
    print(f'model architecture: LinearRegression()')
    print(pearson_df)

(631073, 3) (631073,)
trained on ['K562']
model architecture: LinearRegression()
         train  test  test+diff
K562     0.616 0.589      0.601
HepG2    0.529 0.503      0.546
SK-N-SH  0.574 0.539      0.459
HCT116   0.577 0.559      0.567
A549     0.472 0.431      0.419
(1262146, 3) (1262146,)
trained on ['K562', 'HepG2']
model architecture: LinearRegression()
         train  test  test+diff
K562     0.614 0.585      0.602
HepG2    0.535 0.506      0.554
SK-N-SH  0.575 0.539      0.463
HCT116   0.580 0.560      0.570
A549     0.467 0.426      0.418
(1893219, 3) (1893219,)
trained on ['K562', 'HepG2', 'SK-N-SH']
model architecture: LinearRegression()
         train  test  test+diff
K562     0.614 0.586      0.602
HepG2    0.535 0.506      0.556
SK-N-SH  0.577 0.540      0.468
HCT116   0.580 0.561      0.570
A549     0.468 0.428      0.419


In [13]:
cell_types = ['K562', 'HepG2', 'SK-N-SH', 'HCT116', 'A549']
assays = ['DNase', 'H3K4me3', 'H3K27ac']
data_splits = ['train', 'test', 'test+diff']


# for train_cell_types in [['K562'], ['HepG2'], ['SK-N-SH'], ['K562', 'HepG2'], ['K562', 'HepG2', 'SK-N-SH']]:
for train_cell_types in [['K562'], ['K562', 'HepG2'], ['K562', 'HepG2', 'SK-N-SH']]:
    # train
    train_data_split = 'train'

    X_train = []
    y_train = []
    for cell_type in train_cell_types:
        condition = data_split_dict[train_data_split]
        X_columns = [f'{cell_type}_{assay}' for assay in assays]
        y_column = cell_type
        
        X_train.append(MPRA_df.loc[condition, X_columns])
        y_train.append(MPRA_df.loc[condition, y_column])

    X_train = np.concatenate(X_train, axis=0)
    y_train = np.concatenate(y_train, axis=0)
    print(X_train.shape, y_train.shape)

    model = MLPRegressor(hidden_layer_sizes=20, alpha=1e-3)
    model.fit(X_train, y_train)


    # valid
    pearson_df = pd.DataFrame(dtype=float)

    for cell_type in cell_types:
        for data_split in data_splits:
            condition = data_split_dict[data_split]
            X_columns = [f'{cell_type}_{assay}' for assay in assays]
            y_column = cell_type

            X_valid = np.array(MPRA_df.loc[condition, X_columns])
            y_valid = np.array(MPRA_df.loc[condition, y_column])

            y_pred = model.predict(X_valid)

            r = pearson(y_valid, y_pred)[0]
            pearson_df.loc[cell_type, data_split] = r

    print(f'trained on {train_cell_types}')
    print(f'model architecture: MLPRegressor()')
    print(pearson_df)

(631073, 3) (631073,)
trained on ['K562']
model architecture: MLPRegressor()
         train  test  test+diff
K562     0.664 0.633      0.651
HepG2    0.573 0.546      0.627
SK-N-SH  0.465 0.435      0.394
HCT116   0.579 0.557      0.593
A549     0.384 0.346      0.380
(1262146, 3) (1262146,)
trained on ['K562', 'HepG2']
model architecture: MLPRegressor()
         train  test  test+diff
K562     0.660 0.629      0.642
HepG2    0.595 0.566      0.629
SK-N-SH  0.504 0.476      0.403
HCT116   0.598 0.579      0.605
A549     0.450 0.422      0.489
(1893219, 3) (1893219,)
trained on ['K562', 'HepG2', 'SK-N-SH']
model architecture: MLPRegressor()
         train  test  test+diff
K562     0.646 0.618      0.629
HepG2    0.588 0.558      0.616
SK-N-SH  0.602 0.569      0.508
HCT116   0.600 0.581      0.607
A549     0.479 0.449      0.467
